# Complete Unified Notebook - JUnit Test Generation
## Llama-3.1-8B Fine-Tuning for Software Testing

**Final Model Results (70K AGGRESSIVE):**
- **Precision:** 67.88% ✅
- **Recall:** 75.71% ✅
- **F1-Score:** 69.90% ✅
- **BLEU:** 14.00%

**Model Location:** `/workspace/finetuning/final_model_70K_AGGRESSIVE/`

---

## Notebook Contents:
1. **Environment Setup** - Package installation and GPU check
2. **Data Preparation** - Loading and processing datasets
3. **Model Training** - Complete 70K AGGRESSIVE training
4. **Evaluation** - Testing on 100 samples
5. **Model Merging** - Merge LoRA adapters with base model
6. **Inference** - Test the model on new examples

---

**Note:** This notebook combines all code from the experimental notebook and the successful 70K AGGRESSIVE training.

---
# PART 1: ENVIRONMENT SETUP
---

## 1.1 Package Installation

In [ ]:
# Install required packages
!pip install -q torch transformers peft trl datasets bitsandbytes accelerate nltk matplotlib seaborn

print("✅ Package installation complete!")

## 1.2 Import Libraries and Check GPU

In [ ]:
import gc
import os
import pandas as pd
import torch
import numpy as np
import re
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    BitsAndBytesConfig, 
    LlamaForCausalLM, 
    TrainingArguments
)
from peft import LoraConfig, prepare_model_for_kbit_training, PeftModel, get_peft_model
from trl import SFTTrainer
from datasets import Dataset, concatenate_datasets, DatasetDict, load_from_disk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# Configure Hugging Face cache location
os.environ['HF_HOME'] = '/workspace/finetuning/huggingface'
print(f"✓ Models will be cached in: {os.environ['HF_HOME']}")

# Check GPU
if torch.cuda.is_available():
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️  WARNING: No GPU detected!")

## 1.3 HuggingFace Login

**Required:** You need access to Llama 3.1 model from Meta.
1. Request access: https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct
2. Get your token: https://huggingface.co/settings/tokens

In [ ]:
from huggingface_hub import login

# Enter your HuggingFace token
hf_token = input("Enter your HuggingFace token: ")
login(token=hf_token)

print("✅ Successfully logged in to HuggingFace!")

---
# PART 2: DATA PREPARATION
---

**Note:** We'll use the already prepared `FYP_dataset_70K_ENHANCED` dataset.

If you need to prepare the dataset from scratch, uncomment the cells below.

## 2.1 Load Dataset

Loading the enhanced 70K dataset that was used for the best training.

In [ ]:
# Load the enhanced 70K dataset
print("Loading FYP_dataset_70K_ENHANCED...")
dataset = load_from_disk("FYP_dataset_70K_ENHANCED")

print(f"\nDataset Info:")
print(f"Train samples: {len(dataset['train']):,}")
print(f"Test samples: {len(dataset['test']):,}")

# Show example
print("\n" + "="*80)
print("SAMPLE TRAINING EXAMPLE:")
print("="*80)
example = dataset['train'][0]
for msg in example['messages']:
    print(f"\n[{msg['role'].upper()}]")
    content = msg['content']
    print(content[:300] + "..." if len(content) > 300 else content)

---
# PART 3: MODEL TRAINING (70K AGGRESSIVE)
---

## 3.1 Load Tokenizer

In [ ]:
base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"✅ Tokenizer loaded: {tokenizer.__class__.__name__}")
print(f"Vocab size: {len(tokenizer):,}")
print(f"Pad token: {tokenizer.pad_token}")

## 3.2 Prepare Training Data

In [ ]:
train_data = dataset['train']
eval_data = dataset['test'].select(range(min(500, len(dataset['test']))))

print(f"Train samples: {len(train_data):,}")
print(f"Eval samples: {len(eval_data):,}")

## 3.3 Configure 4-bit Quantization (QLoRA)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print("✅ QLoRA configuration created")

## 3.4 Load Base Model

In [ ]:
print("Loading base model with 4-bit quantization...")
print("(This will download ~16GB on first run)")

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model)

print(f"✅ Model loaded: {model.__class__.__name__}")
print(f"Device: {model.device}")

## 3.5 Configure LoRA

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"]
)

print("✅ LoRA config created:")
print(f"   Rank (r): {peft_config.r}")
print(f"   Alpha: {peft_config.lora_alpha}")
print(f"   Dropout: {peft_config.lora_dropout}")

## 3.6 Training Configuration

In [ ]:
training_args = TrainingArguments(
    output_dir='./results_70K_AGGRESSIVE',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=5e-4,
    lr_scheduler_type='cosine',
    warmup_ratio=0.05,
    logging_steps=10,
    save_steps=250,
    eval_steps=250,
    eval_strategy='steps',
    save_total_limit=3,
    bf16=True,
    optim='paged_adamw_8bit',
    max_grad_norm=0.3,
    group_by_length=True,
    report_to='none',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False
)

total_steps = (len(train_data) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)) * training_args.num_train_epochs
print(f"\nTotal training steps: {total_steps:,}")
print(f"Estimated time: ~6-7 hours\n")

## 3.7 Create Trainer

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field='text',
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_args,
    packing=False
)

print("✅ Trainer initialized successfully!")

## 3.8 Start Training

**⚠️ WARNING:** This will take approximately 6-7 hours!

In [ ]:
print("\n" + "="*80)
print("STARTING 70K AGGRESSIVE TRAINING...")
print("="*80 + "\n")

trainer.train()

print("\n" + "="*80)
print("TRAINING COMPLETED!")
print("="*80)

## 3.9 Save Model

In [ ]:
output_dir = "./final_model_70K_AGGRESSIVE"
print(f"\nSaving model to {output_dir}...")
trainer.save_model(output_dir)

print(f"\n✅ Model saved to: {output_dir}")
print("="*80)

---
# PART 4: EVALUATION
---

## 4.1 Load Trained Model for Evaluation

In [ ]:
print("="*80)
print("70K AGGRESSIVE MODEL EVALUATION")
print("="*80)

# Load tokenizer
print("\nLoading tokenizer...")
eval_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
eval_tokenizer.pad_token = eval_tokenizer.eos_token
eval_tokenizer.padding_side = "left"

# Load base model
print("Loading base model...")
eval_bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

eval_base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    quantization_config=eval_bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# Load LoRA adapter
print("Loading LoRA adapter from ./final_model_70K_AGGRESSIVE...")
eval_model = PeftModel.from_pretrained(eval_base_model, "./final_model_70K_AGGRESSIVE")
eval_model.eval()

print("✅ Model loaded for evaluation!")

## 4.2 Load Test Dataset

In [ ]:
print("\nLoading test dataset...")
eval_dataset = load_from_disk("FYP_dataset_70K_ENHANCED")
test_data = eval_dataset['test']

print(f"Total test samples: {len(test_data):,}")
print("Evaluating on 100 samples...")
num_eval_samples = min(100, len(test_data))
test_samples = test_data.select(range(num_eval_samples))

## 4.3 Define Evaluation Metrics

In [ ]:
def tokenize_code(code):
    """Tokenize code into tokens for comparison."""
    code = re.sub(r'\s+', ' ', code.strip())
    tokens = re.findall(r'\w+|[^\w\s]', code)
    return [t.strip() for t in tokens if t.strip()]

def calculate_metrics(generated, expected):
    """Calculate Precision, Recall, F1-Score."""
    gen_tokens = set(tokenize_code(generated))
    exp_tokens = set(tokenize_code(expected))

    if not gen_tokens:
        return 0.0, 0.0, 0.0

    true_positives = len(gen_tokens & exp_tokens)
    precision = true_positives / len(gen_tokens) if gen_tokens else 0.0
    recall = true_positives / len(exp_tokens) if exp_tokens else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    return precision, recall, f1

def calculate_bleu(generated, expected):
    """Calculate BLEU score."""
    gen_tokens = tokenize_code(generated)
    exp_tokens = tokenize_code(expected)

    if not gen_tokens or not exp_tokens:
        return 0.0

    smoothing = SmoothingFunction().method1
    return sentence_bleu([exp_tokens], gen_tokens, smoothing_function=smoothing)

print("✅ Evaluation metrics defined!")

## 4.4 Run Evaluation

In [ ]:
print("\nStarting evaluation...\n")
total_precision = 0.0
total_recall = 0.0
total_f1 = 0.0
total_bleu = 0.0

for i, example in enumerate(tqdm(test_samples, desc="Evaluating")):
    messages = example['messages']
    prompt_messages = [msg for msg in messages if msg['role'] != 'assistant']

    prompt = eval_tokenizer.apply_chat_template(
        prompt_messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = eval_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=1024)
    inputs = {k: v.to(eval_model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = eval_model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.1,
            do_sample=False,
            pad_token_id=eval_tokenizer.eos_token_id
        )

    generated_text = eval_tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "<|start_header_id|>assistant<|end_header_id|>" in generated_text:
        generated_output = generated_text.split("<|start_header_id|>assistant<|end_header_id|>")[-1].strip()
    else:
        prompt_length = len(eval_tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True))
        generated_output = generated_text[prompt_length:].strip()

    expected_output = messages[-1]['content']

    precision, recall, f1 = calculate_metrics(generated_output, expected_output)
    bleu = calculate_bleu(generated_output, expected_output)

    total_precision += precision
    total_recall += recall
    total_f1 += f1
    total_bleu += bleu

    if i == 0:
        print("\n" + "="*80)
        print("EXAMPLE OUTPUT (Sample 1):")
        print("="*80)
        print("\n--- EXPECTED ---")
        print(expected_output[:500])
        print("\n--- GENERATED ---")
        print(generated_output[:500])
        print("\n--- METRICS FOR THIS SAMPLE ---")
        print(f"Precision: {precision*100:.2f}%")
        print(f"Recall: {recall*100:.2f}%")
        print(f"F1-Score: {f1*100:.2f}%")
        print(f"BLEU: {bleu*100:.2f}%")
        print("="*80 + "\n")

print("\n✅ Evaluation complete!")

## 4.5 Display Results

In [ ]:
avg_precision = (total_precision / num_eval_samples) * 100
avg_recall = (total_recall / num_eval_samples) * 100
avg_f1 = (total_f1 / num_eval_samples) * 100
avg_bleu = (total_bleu / num_eval_samples) * 100

print("\n" + "="*80)
print("FINAL RESULTS - 70K AGGRESSIVE MODEL")
print("="*80)
print(f"\nEvaluated on {num_eval_samples} test samples")
print(f"\nAverage Precision: {avg_precision:.2f}%")
print(f"Average Recall: {avg_recall:.2f}%")
print(f"Average F1-Score: {avg_f1:.2f}%")
print(f"Average BLEU Score: {avg_bleu:.2f}%")
print("="*80)

---
# PART 5: MODEL MERGING
---

**Purpose:** Merge the LoRA adapters with the base model to create a standalone model.

**Note:** This section was NOT run in the previous notebook. Run it if you want a merged model.

## 5.1 Clear GPU Memory

In [ ]:
# Clear previous models from memory
try:
    del model
    del eval_model
    del eval_base_model
except:
    pass

import gc
gc.collect()
torch.cuda.empty_cache()

print("GPU Memory cleared:")
print(f"  Allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
print(f"  Reserved: {torch.cuda.memory_reserved()/1024**3:.2f} GB")

## 5.2 Load Model in FP16 and Merge

In [ ]:
print("="*80)
print("MERGING LORA ADAPTERS WITH BASE MODEL")
print("="*80)

base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
adapter_model = "./final_model_70K_AGGRESSIVE"
merged_model_path = "./final_model_70K_AGGRESSIVE_MERGED"

# Load base model in FP16 (without quantization for merging)
print("\nLoading base model in FP16...")
merge_model = LlamaForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
print("✅ Base model loaded")

# Load LoRA adapter
print(f"\nLoading LoRA adapter from {adapter_model}...")
merge_model = PeftModel.from_pretrained(merge_model, adapter_model)
print("✅ LoRA adapter loaded")

# Merge and unload
print("\nMerging LoRA weights into base model...")
merge_model = merge_model.merge_and_unload()
print("✅ Weights merged!")

# Reload tokenizer
print("\nReloading tokenizer...")
merge_tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
merge_tokenizer.pad_token = merge_tokenizer.eos_token
merge_tokenizer.padding_side = "right"
print("✅ Tokenizer loaded")

# Save merged model
print(f"\nSaving merged model to {merged_model_path}...")
merge_model.save_pretrained(merged_model_path)
merge_tokenizer.save_pretrained(merged_model_path)
print("✅ Merged model saved!")

print("\n" + "="*80)
print("MERGING COMPLETE!")
print("="*80)
print(f"\nMerged model location: {merged_model_path}")
print("\nNote: This is a fully merged model (~16GB) that doesn't need LoRA adapters.")

---
# PART 6: INFERENCE
---

**Purpose:** Test the trained model on new examples.

**Note:** This section was NOT run in the previous notebook. Run it to test your model!

## 6.1 Load Model for Inference

**Choose ONE option:**
- **Option A:** Load LoRA adapter model (smaller, faster)
- **Option B:** Load merged model (if you ran Part 5)

In [ ]:
print("="*80)
print("LOADING MODEL FOR INFERENCE")
print("="*80)

# OPTION A: Load with LoRA adapters (recommended)
use_lora = True  # Set to False to use merged model

if use_lora:
    print("\nLoading model with LoRA adapters...")
    
    # Load tokenizer
    infer_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
    infer_tokenizer.pad_token = infer_tokenizer.eos_token
    infer_tokenizer.padding_side = "right"
    
    # Load base model with quantization
    infer_bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )
    
    infer_model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Meta-Llama-3.1-8B-Instruct",
        quantization_config=infer_bnb_config,
        device_map="auto",
        trust_remote_code=True,
    )
    
    # Load LoRA adapter
    infer_model = PeftModel.from_pretrained(infer_model, "./final_model_70K_AGGRESSIVE")
    infer_model.eval()
    
    print("✅ Model loaded with LoRA adapters!")
    
else:
    print("\nLoading merged model...")
    
    # Load tokenizer
    infer_tokenizer = AutoTokenizer.from_pretrained("./final_model_70K_AGGRESSIVE_MERGED")
    
    # Load merged model
    infer_model = AutoModelForCausalLM.from_pretrained(
        "./final_model_70K_AGGRESSIVE_MERGED",
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True,
    )
    infer_model.eval()
    
    print("✅ Merged model loaded!")

print("\n" + "="*80)
print("MODEL READY FOR INFERENCE!")
print("="*80)

## 6.2 Test on Example Focal Method

In [ ]:
# Example focal method to test
focal_method = """VerificationUtil { 
    static public boolean isZero(Number value, double zeroThreshold){ 
        return (value.doubleValue() >= -zeroThreshold) && (value.doubleValue() <= zeroThreshold); 
    } 
}"""

print("🔧 Generating test case...\n")
print("="*80)
print("INPUT FOCAL METHOD:")
print("="*80)
print(focal_method)
print("="*80)

## 6.3 Generate Test Case

In [ ]:
# Prepare messages in Llama 3.1 format
messages = [
    {
        "role": "system",
        "content": "You are an expert Java developer. Generate JUnit test methods for the given focal method. Include @Test annotation and assertions."
    },
    {
        "role": "user",
        "content": f"Generate a JUnit test method for the following focal method:\n\n{focal_method}"
    }
]

# Apply chat template
prompt = infer_tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# Tokenize
inputs = infer_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
inputs = {k: v.to(infer_model.device) for k, v in inputs.items()}

# Generate
print("\n⏳ Generating (this may take 10-30 seconds)...\n")
with torch.no_grad():
    outputs = infer_model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        pad_token_id=infer_tokenizer.eos_token_id
    )

# Decode
generated_text = infer_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract generated test
if "<|start_header_id|>assistant<|end_header_id|>" in generated_text:
    generated_test = generated_text.split("<|start_header_id|>assistant<|end_header_id|>")[-1].strip()
else:
    generated_test = generated_text[len(prompt):].strip()

print("="*80)
print("GENERATED TEST CASE:")
print("="*80)
print(generated_test)
print("="*80)

## 6.4 Custom Inference Function

Use this function to generate tests for your own focal methods.

In [ ]:
def generate_junit_test(focal_method_code, temperature=0.7, max_tokens=512):
    """
    Generate a JUnit test for the given focal method.
    
    Args:
        focal_method_code (str): The Java focal method code
        temperature (float): Sampling temperature (0.1-1.0)
        max_tokens (int): Maximum tokens to generate
    
    Returns:
        str: Generated JUnit test method
    """
    messages = [
        {
            "role": "system",
            "content": "You are an expert Java developer. Generate JUnit test methods for the given focal method. Include @Test annotation and assertions."
        },
        {
            "role": "user",
            "content": f"Generate a JUnit test method for the following focal method:\n\n{focal_method_code}"
        }
    ]
    
    prompt = infer_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = infer_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(infer_model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = infer_model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            pad_token_id=infer_tokenizer.eos_token_id
        )
    
    generated_text = infer_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    if "<|start_header_id|>assistant<|end_header_id|>" in generated_text:
        generated_test = generated_text.split("<|start_header_id|>assistant<|end_header_id|>")[-1].strip()
    else:
        generated_test = generated_text[len(prompt):].strip()
    
    return generated_test

print("✅ generate_junit_test() function defined!")
print("\nUsage:")
print('test = generate_junit_test(your_focal_method)')
print('print(test)')

## 6.5 Try Your Own Focal Method

In [ ]:
# EDIT THIS: Put your own focal method here
my_focal_method = """
public class StringUtils {
    public static boolean isEmpty(String str) {
        return str == null || str.length() == 0;
    }
}
"""

# Generate test
test = generate_junit_test(my_focal_method)

print("="*80)
print("YOUR FOCAL METHOD:")
print("="*80)
print(my_focal_method)
print("\n" + "="*80)
print("GENERATED TEST:")
print("="*80)
print(test)
print("="*80)

---
# NOTEBOOK COMPLETE!
---

## Summary of What This Notebook Does:

1. ✅ **Environment Setup** - Installs packages, checks GPU
2. ✅ **Data Preparation** - Loads 70K enhanced dataset
3. ✅ **Model Training** - Trains 70K AGGRESSIVE model (6-7 hours)
4. ✅ **Evaluation** - Tests on 100 samples, calculates metrics
5. ✅ **Model Merging** - Merges LoRA adapters with base model
6. ✅ **Inference** - Tests model on new focal methods

## Key Results:
- **Precision:** 67.88%
- **Recall:** 75.71%
- **F1-Score:** 69.90%
- **BLEU:** 14.00%

## Model Locations:
- LoRA Adapters: `./final_model_70K_AGGRESSIVE/`
- Merged Model: `./final_model_70K_AGGRESSIVE_MERGED/` (if Part 5 was run)

---

**Ready for deployment!** 🎉